# Sentiment Polarity Prediction with BERT

This notebook contains a basic implementation of document-level sentiment analysis
for movie reviews with BERT using the
Hugging Face [Transformers](https://huggingface.co/transformers/) library
with
[PyTorch](https://pytorch.org/)
and
[Lightning](https://www.pytorchlightning.ai/).


## Data Set

We use the movie review polarity data set of Pang and Lee 2004 [A Sentimental Education: Sentiment Analysis Using Subjectivity Summarization Based on Minimum Cuts](https://www.aclweb.org/anthology/P04-1035/) in Version 2.0 available from http://www.cs.cornell.edu/People/pabo/movie-review-data (section "Sentiment polarity datasets"). This dataset contains 1000 positive and 1000 negative reviews, each tokenised, sentence-split (one sentence per line) and lowercased. Each review has been assigned to 1 of 10 cross-validation folds by the authors and this setup should be followed to compare with published results.

First, we re-use the code from the sentiment polarity prediction with naive Bayes to load the pre-tokenised text.

In [ ]:
import os

# choose between 'local-tgz' and 'web',
# see description under each heading below

data_source = 'local-tgz'

# adjust path as needed

data_tgz    = os.path.join('data', 'pang-and-lee-2004', 'review_polarity.tar.gz')

data_url = 'https://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz'

In [ ]:
#from collections import defaultdict

class PL04DataLoader_Part_1:
    
    def __init__(self):
        pass
    
    def get_labelled_dataset(self, fold = 0):
        ''' Compile a fold of the data set
        '''
        dataset = []
        for label in ('pos', 'neg'):
            for document in self.get_documents(
                fold = fold,
                label = label,
            ):
                dataset.append((document, label))
        return dataset
    
    def get_documents(self, fold = 0, label = 'pos'):
        ''' Enumerate the raw contents of selected data set files.
            Args:
                data_dir: relative or absolute path to the data set folder
                fold: which fold to load (0 to n_folds-1)
                label: 'pos' or 'neg' to
                    select data with positive or negative sentiment
                    polarity
            Return:
                List of tokenised documents, each a list of sentences
                that in turn are lists of tokens
        '''
        raise NotImplementedError

In [ ]:
class PL04DataLoader(PL04DataLoader_Part_1):
    
    def get_xval_splits(self):
        ''' Split data with labels for cross-validation
            returns a list of k pairs (training_data, test_data)
            for k cross-validation
        '''
        # load the folds
        folds = []
        for i in range(10):
            folds.append(self.get_labelled_dataset(
                fold = i
            ))
        # create training-test splits
        retval = []
        for i in range(10):
            test_data = folds[i]
            training_data = []
            for j in range(9):
                ij1 = (i+j+1) % 10
                assert ij1 != i
                training_data = training_data + folds[ij1]
            retval.append((training_data, test_data))
        return retval

In [ ]:
import tarfile
import time

class PL04DataLoaderFromStream(PL04DataLoader):
        
    def __init__(self, tgz_stream, **kwargs):
        super().__init__(**kwargs)
        self.data = {}
        counter = 0
        with tarfile.open(
            mode = 'r|gz',
            fileobj = tgz_stream
        ) as tar_archive:
            for tar_member in tar_archive:
                if counter == 2000:
                    break
                path_components = tar_member.name.split('/')
                filename = path_components[-1]
                if filename.startswith('cv') \
                and filename.endswith('.txt') \
                and '_' in filename:
                    label = path_components[-2]
                    fold = int(filename[2])
                    key = (fold, label)
                    if key not in self.data:
                        self.data[key] = []
                    f = tar_archive.extractfile(tar_member)
                    document = [
                        line.decode('utf-8').split()
                        for line in f.readlines()
                    ]
                    self.data[key].append(document)
                    counter += 1
            
    def get_documents(self, fold = 0, label = 'pos'):
        return self.data[(fold, label)]

## Read Data from the Web
This should run efficiently both on google colab and locally but has the disadvantage that the same data is downloaded each time the notebook is run.

In [ ]:
import urllib.request

class PL04DataLoaderFromURL(PL04DataLoaderFromStream):
    
    def __init__(self, data_url, **kwargs):
        with urllib.request.urlopen(data_url) as tgz_stream:
            super().__init__(tgz_stream, **kwargs)

## Read Data from a Local .tgz File

You manually download the .tgz once to a filesystem that can be accessed from the notebook, e.g. google drive on colab, and this notebook reads this file in one chunk.

In [ ]:
class PL04DataLoaderFromTGZ(PL04DataLoaderFromStream):
    
    def __init__(self, data_path, **kwargs):
        with open(data_path, 'rb') as tgz_stream:
            super().__init__(tgz_stream, **kwargs)

In [ ]:
# load data

if data_source == 'local-tgz':
    data_loader = PL04DataLoaderFromTGZ(data_tgz)
elif data_source == 'web':
    data_loader = PL04DataLoaderFromURL(data_url)
else:
    raise ValueError('Unsupported data source %r' %data_source)

See the notebook on sentiment analysis with naive Bayes for code to test and demonstrate above functionality and to load data from the individual files in the tgz.

## BERT Tokenisation
While the P&L 04 corpus is already tokenised into words and punctuation, BERT expects a tokenisation into subword units and popular BERT libraries provide tools for that.

In [1]:
# adjust the torch version below following instructions on https://pytorch.org/get-started/locally/

import sys

# for why we use {sys.executable} see
# https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/

try:
    import torch
except ModuleNotFoundError:
    !{sys.executable} -m pip install torch==1.7.1+cpu torchvision==0.8.2+cpu torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

try:
    import transformers
except ModuleNotFoundError:
    !{sys.executable} -m pip install transformers

try:
    import pytorch_lightning as pl
except ModuleNotFoundError:
    !{sys.executable} -m pip install pytorch-lightning

try:
    import torchnlp
except ModuleNotFoundError:
    !{sys.executable} -m pip install pytorch-nlp

try:
    import tensorboard
except ModuleNotFoundError:
    !{sys.executable} -m pip install tensorboard

print('Ready')

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Created wheel for sacremoses: filename=sacremoses-0.0.44-py3-none-any.whl size=886084 sha256=cccfa282d84fc49b714a3eb5c04e00f37de1706cada298ac9ef35793d5812267
  Stored in directory: c:\users\owner\appdata\local\pip\cache\wheels\25\9e\71\a1977fb981636b67758f23b683c5acee3902fbd547dcd2f0dd
Successfully built sacremoses
  Using cached future-0.18.2.tar.gz (829 kB)
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.1.0-py2.py3-none-any.whl (147 kB)
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=83d0a9b7ed1f358929c6a6be5448b658b57a3ae389ce727a538037500058b0ab
  Stored in directory: c:\users\owner\appdata\local\pip\cache\wheels\2f\a0\d3\4030d9f80e6b3be787f19fc911b8e7aa462986a40ab1e4bb94
Successfully built future


Ready


In [ ]:
from transformers import AutoTokenizer
from tokenizers.pre_tokenizers import Whitespace

model_size = 'base'  # choose between 'tiny', 'base' and 'large'

size2name = {
    'tiny':  'distilbert-base-uncased',  # TODO: This doesn't seem to reduce memory usage compared to bert-base. 
    'base':  'bert-base-uncased',
    'large': 'bert-large-uncased',
}


model_name = size2name[model_size]

force_whitespace_pre_tokeniser = False  # should not matter with pre-tokenised P&L'04 input

tokeniser = AutoTokenizer.from_pretrained(model_name)
    
if force_whitespace_pre_tokeniser:
    tokeniser.pre_tokenizer = Whitespace()

In [ ]:
# let's test the tokeniser with a small example

example_batch = [
    'hello world !'.split(),
    """tokenisation 's fun""".split(),
]

tokenised_text = tokeniser(
    example_batch,  # pre-tokenised input
    is_split_into_words = True,
)

for i, token_ids in enumerate(tokenised_text['input_ids']):
    print(i, "\t['input_ids']:", token_ids)
    print(   '\ttokens:       ', tokeniser.convert_ids_to_tokens(token_ids))
    print(   "\t.word_ids():  ", tokenised_text.word_ids(batch_index = i))
    example = example_batch[i]
    for token in example:
        # https://stackoverflow.com/questions/62317723/tokens-to-words-mapping-in-the-tokenizer-decode-step-huggingface
        print('%18s --> %r' %(
            token,
            tokeniser.encode(
                token,
                add_special_tokens = False,
            )
        ))

0 	['input_ids']: [101, 7592, 2088, 999, 102]
	tokens:        ['[CLS]', 'hello', 'world', '!', '[SEP]']
	.word_ids():   [None, 0, 1, 2, None]
             hello --> [7592]
             world --> [2088]
                 ! --> [999]
1 	['input_ids']: [101, 19204, 6648, 1005, 1055, 4569, 102]
	tokens:        ['[CLS]', 'token', '##isation', "'", 's', 'fun', '[SEP]']
	.word_ids():   [None, 0, 0, 1, 1, 2, None]
      tokenisation --> [19204, 6648]
                's --> [1005, 1055]
               fun --> [4569]


## Sequence Length Distribution
The BERT models made available to the public have been trained only up to a length of 512 subword units as memory requirements increase quadratically with the sequence length. We will restrict the sequence length further to 256 to keep resource requirements acceptable for mid-range GPUs. 

If many of our documents are longer than that it may pay off to look for a solution to using the full documents with BERT. For an overview of methods, see section 3.3.2 of
 * Palakh Mignonne Jude (2020). Increasing Accessibility of Electronic Theses and Dissertations (ETDs) Through Chapter-level Classification. PhD Thesis. Virginia Polytechnic Institute and State University, Blacksburg, Virginia, USA.

In [ ]:
# if you have sufficient TPU or GPU memory you can increase the sequence length up to 512
# (memory requirements will be the highest during fine-tuning of BERT)
max_sequence_length  = 256

# memory requirements increase linearly with the batch size;
# a batch size of 16, better 32, is needed for efficient training;
# however, with little memory, we have to go lower
batch_size      = 10   # for a 12 GB card

In [ ]:
from collections import defaultdict
    
bin_width = 250

distribution = defaultdict(lambda: 0)

# interate all reviews
for fold in range(10):
    for label in 'pos neg'.split():
        batch = []
        for document in data_loader.get_documents(
            label = label,
            fold = fold,
        ):
            tokens = []
            for sentence in document:
                for token in sentence:
                    tokens.append(token)
            batch.append(tokens)
        tokenised_batch = tokeniser(
            batch,  # pre-tokenised input
            is_split_into_words = True,
        )
        max_length_bin = 0
        for token_ids in tokenised_batch['input_ids']:
            length = len(token_ids)
            length_bin = length // bin_width
            distribution[(label,   length_bin)] += 1
            distribution[('total', length_bin)] += 1
            if length_bin > max_length_bin:
                max_length_bin = length_bin
print('LengthBin\tPos\tNeg\tTotal')
for length_bin in range(0, max_length_bin+1):
    row = []
    row.append('%4d-%4d' %(
        bin_width*length_bin,
        bin_width*(1+length_bin)-1
    ))
    for label in 'pos neg total'.split():
        count = distribution[(label, length_bin)]
        row.append('%d' %count)        
    print('\t'.join(row))
            

Token indices sequence length is longer than the specified maximum sequence length for this model (926 > 512). Running this sequence through the model will result in indexing errors


LengthBin	Pos	Neg	Total
   0- 249	7	13	20
 250- 499	126	148	274
 500- 749	271	317	588
 750- 999	275	298	573
1000-1249	171	136	307
1250-1499	75	49	124
1500-1749	37	20	57
1750-1999	21	10	31


Conclusion: Truncating the documents to the sequence length of the model limits the amount of information available from each document. The following list some options for how we truncate the documents in order of complexity. 
1. Use the start of each document only, i.e. truncate the reviews. Using only the start of each review is also implemented in [Huggingface's example for fine-tuning on custom datasets](https://huggingface.co/transformers/custom_datasets.html).
2. Use the end of each document only, i.e. take a slice from the end.
3. Combine two slices of the document, one from the start and one from the end. What fraction of the sequence to use for the first slice is a parameter of this method. Setting the parameter to 0 or 1 yields methods 1 and 2 above.

## PyTorch DataLoader

* Dataset objects behave like a list of dictionaries, one dictionary for each data instance (training or test item), with user-defined keys.
* DataLoader objects shuffle data provided by a Dataset object and create batches of data.
* LightningDataModule objects create DataLoader objects for training, validation and test data.

In [ ]:
# basic usage of pytorch and lightning from
# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
# and
# https://github.com/ricardorei/lightning-text-classification/blob/master/classifier.py
    
from torch.utils.data import Dataset, DataLoader, RandomSampler

class SlicedDocuments_part_1(Dataset):
    
    def __init__(
        self,
        raw_data,
        tokeniser = None,
        fraction_for_first_sequence = 0.5,
        max_sequence_length = 128,
        second_part_as_sequence_B = False,
        preproc_batch_size = 8,
        
    ):
        '''Extracts slices from labelled documents
           Args:
               raw_data: list of (document, label) pairs
               tokeniser: transformer tokeniser to obtain subword units; this tokeniser
                   will be used to decide how many words to include in each slice; it
                   is recommended to use the same tokeniser that will be used to tokenise
                   the data
               fraction_for_first_sequence: 1 = take slice from the start of the document,
                   0 = take slice from the end of the document, > 0 and < 1 = take two
                   slices, one of this relative size from the start and then fill to
                   max_sequence_length from the end of the document
               max_sequence_length: produce sequences up to this length
               allow_partial_tokens: whether to slice between subword units of tokens; if
                   set to True the produced sequences will always have max_sequence_length
                   items unless the document is very short
        '''
        assert max_sequence_length >= 5
        self.max_sequence_length = max_sequence_length
        self.second_part_as_sequence_B = second_part_as_sequence_B
        self.init_sequence_lengths(fraction_for_first_sequence)
        self.tokeniser = tokeniser
        self.init_slices(raw_data, preproc_batch_size)

    def init_sequence_lengths(self, fraction_for_first_sequence):
        available_for_two_sequences = self.max_sequence_length - 3
        self.first_sequence_length = max(0, int(
            fraction_for_first_sequence * available_for_two_sequences
        ))
        self.last_sequence_length = max(
            0,
            available_for_two_sequences - self.first_sequence_length
        )
        assert self.first_sequence_length \
            + self.last_sequence_length   \
            <= available_for_two_sequences
        if self.first_sequence_length == 0:
            self.last_sequence_length += 1
        elif self.last_sequence_length == 0:
            self.first_sequence_length += 1
        if self.first_sequence_length == 0 \
        or self.last_sequence_length == 0:
            # do not use a second [SEP] marker when there is
            # always only one sequence
            self.second_part_as_sequence_B = False

In [ ]:
class SlicedDocuments_part_2(SlicedDocuments_part_1):
    
    def init_slices(self, raw_data, preproc_batch_size):
        self.slices = []
        next_batch = []
        next_labels = []
        for document, label in raw_data:
            tokens = []
            for sentence in document:
                tokens = tokens + sentence
            next_batch.append(tokens)    
            next_labels.append(label)
            if len(next_batch) >= preproc_batch_size:
                self.add_batch(next_batch, next_labels)
                next_batch = []
                next_labels = []
        if next_batch:
            self.add_batch(next_batch, next_labels)

In [ ]:
class SlicedDocuments_part_3(SlicedDocuments_part_2):
    
    def add_batch(self, batch, labels):
        # determine, for each document in the batch, how many
        # tokens to include from the start of the document
        if self.first_sequence_length:
            lengths_1 = self.get_lengths(batch)
        else:
            lengths_1 = len(batch) * [0]
        # determine, for each document in the batch, how many
        # tokens to include from the end of the document    
        if self.last_sequence_length:
            lengths_2 = self.get_lengths(batch, part = 2, lengths_1 = lengths_1)
        else:
            lengths_2 = len(batch) * [0]
        # TODO: In an earlier version, we did not check the following
        #       condition, creating a second sequence for short
        #       documents even though only one sequence is requested.
        #       First results indicate that this bug actually
        #       improves performance. Future work should investigate
        #       this and, if this effect is confirmed, propose a
        #       clean solution to exploit this effect.
        if self.first_sequence_length:
            # sometimes there is space for more tokens from the
            # start even though no more from the end fit
            lengths_1 = self.expand_lengths(batch, lengths_1, lengths_2)
        # TODO: For cases with length_1 + length_2 > len(tokens),
        #       should we adjust the parts to not overlap?       
        # prepare texts
        for batch_idx, tokens in enumerate(batch):
            parts = []
            length_1 = lengths_1[batch_idx]
            length_2 = lengths_2[batch_idx]
            part_1 = tokens[:length_1]
            if length_2 > 0:
                part_2 = tokens[-length_2:]
            else:
                part_2 = []
            if self.second_part_as_sequence_B:
                parts.append(part_1)
                parts.append(part_2)
            else:
                parts.append(part_1 + part_2)
            assert len(parts) > 0    
            self.slices.append((parts, labels[batch_idx]))

In [ ]:
class SlicedDocuments_part_4(SlicedDocuments_part_3):

    def get_lengths(
        self, batch, part = 1, lengths_1 = None,
        lengths_2 = None,
    ):
        if part == 3:
            lower_limits = lengths_1[:]  # clone
        else:
            lower_limits = len(batch) * [0]
        upper_limits = []
        for tokens in batch:
            upper_limits.append(min(
                len(tokens),
                self.max_sequence_length
            ))
        # we want each upper limit to be a sequence length
        # that is too big but sometimes the full document
        # (or max_length words) can fit --> test for this
        # special case
        for batch_idx, fit in enumerate(
            self.get_fit(batch, upper_limits, part, lengths_1, lengths_2)
        ):
            if fit:
                # update lower limit to match upper limit
                # to mark this document as not needing any
                # further length search
                lower_limits[batch_idx] = upper_limits[batch_idx]
        while True:
            # prepare next lengths to test and check whether search is finished
            new_limits = []
            all_done = True
            for batch_idx, tokens in enumerate(batch):
                if lower_limits[batch_idx]+1 >= upper_limits[batch_idx]:
                    new_limits.append(lower_limits[batch_idx])
                else:
                    all_done = False
                    new_limits.append((
                        lower_limits[batch_idx] + upper_limits[batch_idx]
                    )//2)
            if all_done:
                return lower_limits
            # adjust lower and upper limits
            for batch_idx, fit in enumerate(
                self.get_fit(batch, new_limits, part, lengths_1, lengths_2)
            ):
                if fit:
                    lower_limits[batch_idx] = new_limits[batch_idx]
                else:
                    upper_limits[batch_idx] = new_limits[batch_idx]

In [ ]:
class SlicedDocuments_part_5(SlicedDocuments_part_4):

    def get_fit(self, batch, limits, part, lengths_1, lengths_2 = None):
        sliced_batch_A = []
        sliced_batch_B = []
        for batch_idx, tokens in enumerate(batch):
            if part == 1:
                length_1 = limits[batch_idx]
                length_2 = 0
            elif part == 2:
                length_1 = lengths_1[batch_idx]
                length_2 = limits[batch_idx]
            else:
                length_1 = limits[batch_idx]
                length_2 = lengths_2[batch_idx]
            part_1 = tokens[:length_1]
            if length_2 > 0:
                part_2 = tokens[-length_2:]
            else:
                part_2 = []
            if self.second_part_as_sequence_B:
                sliced_batch_A.append(part_1)
                sliced_batch_B.append(part_2)
            else:
                sliced_batch_A.append(part_1 + part_2)
        if self.second_part_as_sequence_B:
            tokenised = self.tokeniser(
               sliced_batch_A, sliced_batch_B,
               is_split_into_words = True,
            )
        else:
            tokenised = self.tokeniser(
               sliced_batch_A,
               is_split_into_words = True,
            )
        # check lengths in subword pieced
        retval = []
        for batch_idx, subword_ids in enumerate(tokenised['input_ids']):
            if part == 1:
                length = len(subword_ids) - 2  # account for [CLS] and [SEP]
                if self.second_part_as_sequence_B:
                    length -= 1                # account for second [SEP] token
                fit = length <= self.first_sequence_length
            else:
                fit = len(subword_ids) <= self.max_sequence_length
            retval.append(fit)
        return retval

In [ ]:
class SlicedDocuments(SlicedDocuments_part_5):

    def __len__(self):
        return len(self.slices)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            assert isinstance(idx, int)
        parts, label = self.slices[idx]
        retval = {}
        retval['parts'] = parts
        retval['label'] = label
        return retval
    
    def expand_lengths(self, batch, lengths_1, lengths_2):
        ''' pushes lengths_1 as far out as possible '''
        return self.get_lengths(
            batch, part = 3,
            lengths_1 = lengths_1,
            lengths_2 = lengths_2,
        )

## Create Training-Test Splits for Cross-Validation

In [ ]:
splits = data_loader.get_xval_splits()
print('Ready')

Ready


## Test Above Dataset Class
Test Slicing of Documents to BERT Sequence Length

In [ ]:
# this may take a minute

raw_data = splits[0][0]
sliced_docs = SlicedDocuments(
    raw_data, tokeniser,
    fraction_for_first_sequence = 0.25,
    max_sequence_length = max_sequence_length, # 99,
    second_part_as_sequence_B = True,
)
print('%d training documents ready in x-val fold 0' %len(sliced_docs))

1800 training documents ready in x-val fold 0


In [ ]:
length2count = defaultdict(lambda: 0)

print('doc_idx seq_len tokens1 tokens2 total')
for doc_idx, sliced_doc in enumerate(sliced_docs):
    parts = sliced_doc['parts']
    if len(parts) == 2:
        tokenised = tokeniser(
           [parts[0]],
           [parts[1]],
           is_split_into_words = True,
        )
        tokens1 = len(parts[0])
        tokens2 = len(parts[1])
    else:
        tokenised = tokeniser(
           [parts[0]],
           is_split_into_words = True,
        )
        tokens1 = len(parts[0])
        tokens2 = 0
    length = len(tokenised['input_ids'][0])
    length2count[length] += 1             

    if length < 90:
        print(
            '\nFirst and last 3 sentences of doc_idx', doc_idx,
            'with', length, 'seq_len:'
        )
        sent_idx = set()
        for idx in range(3):
            sent_idx.add(idx)
            idx = len(raw_data[doc_idx][0]) - 1 - idx
            if idx >= 0:
                sent_idx.add(idx)
        for s_idx in sorted(list(sent_idx)):
            try:
                sentence = raw_data[doc_idx][0][s_idx]
                print('[%d] %r' %(s_idx, sentence))
            except IndexError:
                pass
        print('Selected slice 1:', parts[0])
        if len(parts) == 2:
            print('Selected slice 2:', parts[1])
    if doc_idx < 10:
        print('%7d %7d %7d %7d %5d' %(
            doc_idx, length, tokens1, tokens2, tokens1+tokens2,
        ))
        
print('\nFrequency of each sequence length:')        
for length in sorted(list(length2count.keys())):
    print(length, length2count[length])

doc_idx seq_len tokens1 tokens2 total
      0     256      59     175   234
      1     256      59     159   218
      2     256      50     167   217
      3     256      53     178   231
      4     256      46     177   223
      5     256      50     150   200
      6     256      61     148   209
      7     256      58     171   229
      8     256      51     174   225
      9     256      48     146   194

First and last 3 sentences of doc_idx 906 with 53 seq_len:
[0] ['this', 'film', 'is', 'extraordinarily', 'horrendous', 'and', "i'm", 'not', 'going', 'to', 'waste', 'any', 'more', 'words', 'on', 'it', '.']
Selected slice 1: ['this', 'film', 'is', 'extraordinarily', 'horrendous', 'and', "i'm", 'not', 'going', 'to', 'waste', 'any', 'more', 'words', 'on', 'it', '.']
Selected slice 2: ['this', 'film', 'is', 'extraordinarily', 'horrendous', 'and', "i'm", 'not', 'going', 'to', 'waste', 'any', 'more', 'words', 'on', 'it', '.']

Frequency of each sequence length:
53 1
243 1
254 5
255

In [ ]:
# https://github.com/ricardorei/lightning-text-classification/blob/master/classifier.py
    
import pytorch_lightning as pl
from torchnlp.encoders import LabelEncoder

class SlicedDataModule_Part_1(pl.LightningDataModule):
    
    def __init__(self, classifier, data_split = None, **kwargs):
        super().__init__()
        self.hparams = classifier.hparams
        self.classifier = classifier
        if data_split is None:
            # this happens when loading a checkpoint
            data_split = (None, None, None)
        elif len(data_split) == 2:
            # add empty validation set
            tr_data, val_data = self.split(data_split[0], 0.9)
            data_split = (tr_data, val_data, data_split[1])
        self.data_split = data_split
        self.kwargs = kwargs
        self.label_encoder = LabelEncoder(
            ['pos', 'neg'],
            reserved_labels = [],
        )

    def train_dataloader(self) -> DataLoader:
        assert self.hparams.batch_size <= batch_size
        dataset = SlicedDocuments(
            raw_data = self.data_split[0],
            **self.kwargs
        )
        return DataLoader(
            dataset = dataset,
            sampler     = RandomSampler(dataset),
            batch_size  = self.hparams.batch_size,
            collate_fn  = self.classifier.prepare_sample,
            num_workers = self.hparams.loader_workers,
        )
  

In [ ]:
class SlicedDataModule(SlicedDataModule_Part_1):
    
    def val_dataloader(self) -> DataLoader:
        assert self.hparams.batch_size <= batch_size
        if not self.data_split[1]:
            return None   # TODO: check documentation what to return
        return DataLoader(
            dataset = SlicedDocuments(
                raw_data = self.data_split[1],
                **self.kwargs
            ),
            batch_size  = self.hparams.batch_size,
            collate_fn  = self.classifier.prepare_sample,
            num_workers = self.hparams.loader_workers,
        )
    
    def test_dataloader(self) -> DataLoader:
        assert self.hparams.batch_size <= batch_size
        return DataLoader(
            dataset = SlicedDocuments(
                raw_data = self.data_split[2],
                **self.kwargs
            ),
            batch_size  = self.hparams.batch_size,
            collate_fn  = self.classifier.prepare_sample,
            num_workers = self.hparams.loader_workers,
        )
    
    def split(self, data, ratio):
        # get label distribution:
        distribution = defaultdict(lambda: 0)
        for _, label in data:
            distribution[label] += 1
        # get target frequencies of labels in first set
        still_needed = defaultdict(lambda: 0)
        for label in distribution:
            still_needed[label] = int(ratio*distribution[label])
        # split data accordingly
        dataset_1 = []
        dataset_2 = []
        for item in data:
            label = item[1]
            if still_needed[label] > 0:
                dataset_1.append(item)
                still_needed[label] -= 1
            else:
                dataset_2.append(item)
        return dataset_1, dataset_2

In [ ]:
from transformers import AutoModel
import torch.nn as nn

class Classifier_part_1(pl.LightningModule):
    
    def __init__(self, hparams = None, **kwargs) -> None:
        super().__init__()
        if type(hparams) is dict:
            #print('Converting', type(hparams))
            hparams = pl.utilities.AttributeDict(hparams)
        #print('New classifier with', hparams)
        self.hparams = hparams
        self.batch_size = hparams.batch_size
        self.data = SlicedDataModule(self, **kwargs)
        if 'tokeniser' in kwargs:
            self.tokenizer = kwargs['tokeniser']  # attribute expected by lightning
        else:
            # this happens when loading a checkpoint
            self.tokenizer = None  # TODO: this may break ability to use the model
        self.__build_model()
        self.__build_loss()
        if hparams.nr_frozen_epochs > 0:
            self.freeze_encoder()
        else:
            self._frozen = False
        self.nr_frozen_epochs = hparams.nr_frozen_epochs
        self.record_predictions = False
            
    def __build_model(self) -> None:
        ''' Init BERT model, tokeniser and classification head '''
        # Q: Why not use AutoModelForSequenceClassification?
        self.bert = AutoModel.from_pretrained(
            model_name,  # was: self.hparams.encoder_model
            output_hidden_states = True
        )
        self.classification_head = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.bert.config.hidden_size, 1536),
            nn.Tanh(),
            nn.Dropout(0.5),
            nn.Linear(1536, 256),
            nn.Tanh(),
            nn.Dropout(0.1),
            nn.Linear(256, self.data.label_encoder.vocab_size)
        )
        
    def __build_loss(self):
        self._loss = nn.CrossEntropyLoss()

In [ ]:
import logging as log

class Classifier_part_2(Classifier_part_1):
    
    def unfreeze_encoder(self) -> None:
        if self._frozen:
            log.info('\n== Encoder model fine-tuning ==')
            for param in self.bert.parameters():
                param.requires_grad = True
            self._frozen = False
            
    def freeze_encoder(self) -> None:
        for param in self.bert.parameters():
            param.requires_grad = False
        self._frozen = True

    def predict(self, sample: dict) -> dict:
        if self.training:
            self.eval()
        with torch.no_grad():
            batch_inputs, _ = self.prepare_sample(
                [sample],
                prepare_target = False
            )
            model_out = self.forward(batch_inputs)
            logits = torch.Tensor.cpu(model_out["logits"]).numpy()
            predicted_labels = [
                self.data.label_encoder.index_to_token[prediction]
                for prediction in numpy.argmax(logits, axis=1)
            ]
            sample["predicted_label"] = predicted_labels[0]
        return sample
    
    def start_recording_predictions(self):
        self.record_predictions = True
        self.reset_recorded_predictions()
        
    def stop_recording_predictions(self):
        self.record_predictions = False
        
    def reset_recorded_predictions(self):
        self.seq2label = {}

In [ ]:
from torchnlp.utils import lengths_to_mask

class Classifier_part_3(Classifier_part_2):
    
    def forward(self, batch_input):
        tokens  = batch_input['input_ids']
        lengths = batch_input['length']
        mask = batch_input['attention_mask']
        # Run BERT model.
        word_embeddings = self.bert(tokens, mask).last_hidden_state
        sentemb = word_embeddings[:,0]  # at position of [CLS]
        logits = self.classification_head(sentemb)
        # Hack to conveniently use the model and trainer to
        # get predictions for a test set:
        if self.record_predictions:
            logits_np = torch.Tensor.cpu(logits).numpy()
            predicted_labels = [
                self.data.label_encoder.index_to_token[prediction]
                for prediction in numpy.argmax(logits_np, axis=1)
            ]
            for index, input_token_ids in enumerate(tokens):
                key = torch.Tensor.cpu(input_token_ids).numpy().tolist()
                # truncate trailing zeros
                while key and key[-1] == 0:
                    del key[-1]
                self.seq2label[tuple(key)] = predicted_labels[index]
        return {"logits": logits}
    
    def loss(self, predictions: dict, targets: dict) -> torch.tensor:
        """
        Computes Loss value according to a loss function.
        :param predictions: model specific output. Must contain a key 'logits' with
            a tensor [batch_size x 1] with model predictions
        :param labels: Label values [batch_size]
        Returns:
            torch.tensor with loss value.
        """
        return self._loss(predictions["logits"], targets["labels"])

In [ ]:
class Classifier_part_4(Classifier_part_3):
    
    def prepare_sample(self, sample: list, prepare_target: bool = True) -> (dict, dict):
        """
        Function that prepares a sample to input the model.
        :param sample: list of dictionaries.
        
        Returns:
            - dictionary with the expected model inputs.
            - dictionary with the expected target labels.
        """
        assert len(sample) <= batch_size
        assert self.tokenizer is not None
        with_1_part = 0
        with_2_parts = 0
        batch_part_1 = []
        batch_part_2 = []
        for item in sample:
            parts = item['parts']
            if len(parts) == 2:
                with_2_parts += 1
                batch_part_1.append(parts[0])
                batch_part_2.append(parts[1])
            else:
                with_1_part += 1
                batch_part_1.append(parts[0])
        assert not (with_1_part and with_2_parts)
        kwargs = {
            'is_split_into_words': True,
            'return_length':       True,
            'padding':             'max_length',
            # https://github.com/huggingface/transformers/issues/8691
            'return_tensors':      'pt',
        }
        if with_2_parts:
            encoded_batch = self.tokenizer(
                batch_part_1,
                batch_part_2,
                **kwargs
            )
        else:
            encoded_batch = self.tokenizer(
                batch_part_1,
                **kwargs
            )
        if not prepare_target:
            return encoded_batch, {}
        # Prepare target:
        batch_labels = []
        for item in sample:
            batch_labels.append(item['label'])
        assert len(batch_labels) <= batch_size
        try:
            targets = {
                "labels": self.data.label_encoder.batch_encode(batch_labels)
            }
            return encoded_batch, targets
        except RuntimeError:
            raise Exception("Label encoder found an unknown label.")

In [ ]:
from collections import OrderedDict

class Classifier_part_5(Classifier_part_4):
    
    def training_step(self, batch: tuple, batch_nb: int, *args, **kwargs) -> dict:
        inputs, targets = batch
        model_out = self.forward(inputs)
        loss_val = self.loss(model_out, targets)
        # in DP mode (default) make sure if result is scalar, there's another dim in the beginning
        # Q: What is this about?
        if self.trainer.use_dp or self.trainer.use_ddp2:
            loss_val = loss_val.unsqueeze(0)
        output = OrderedDict({"loss": loss_val})
        self.log('train_loss', loss_val, on_step=True, on_epoch=True, prog_bar=True)
        # can also return just a scalar instead of a dict (return loss_val)
        return output
   
    def test_or_validation_step(self, test_type, batch: tuple, batch_nb: int, *args, **kwargs) -> dict:
        inputs, targets = batch
        model_out = self.forward(inputs)
        loss_val = self.loss(model_out, targets)
        y = targets["labels"]
        y_hat = model_out["logits"]
        # acc
        labels_hat = torch.argmax(y_hat, dim=1)
        val_acc = torch.sum(y == labels_hat).item() / (len(y) * 1.0)
        val_acc = torch.tensor(val_acc)
        if self.on_gpu:
            val_acc = val_acc.cuda(loss_val.device.index)
        # in DP mode (default) make sure if result is scalar, there's another dim in the beginning
        if self.trainer.use_dp or self.trainer.use_ddp2:
            loss_val = loss_val.unsqueeze(0)
            val_acc = val_acc.unsqueeze(0)
        output = OrderedDict({
            test_type + "_loss": loss_val,
            test_type + "_acc":  val_acc,
            'batch_size': len(batch),
            #'predictions': labels_hat,
        })
        return output
    
    def validation_step(self, batch: tuple, batch_nb: int, *args, **kwargs) -> dict:
        return self.test_or_validation_step(
            'val', batch, batch_nb, *args, **kwargs
        )
    
    def test_step(self, batch: tuple, batch_nb: int, *args, **kwargs) -> dict:
        return self.test_or_validation_step(
            'test', batch, batch_nb, *args, **kwargs
        )

In [ ]:
from torch import optim

class Classifier(Classifier_part_5):
    
    # validation_end() is now validation_epoch_end()
    # https://github.com/PyTorchLightning/pytorch-lightning/blob/efd272a3cac2c412dd4a7aa138feafb2c114326f/CHANGELOG.md
    
    def test_or_validation_epoch_end(self, test_type, outputs: list) -> None:
        val_loss_mean = 0.0
        val_acc_mean = 0.0
        total_size = 0
        for output in outputs:
            val_loss = output[test_type + "_loss"]
            # reduce manually when using dp
            if self.trainer.use_dp or self.trainer.use_ddp2:
                val_loss = torch.mean(val_loss)
            val_loss_mean += val_loss
            # reduce manually when using dp
            val_acc = output[test_type + "_acc"]
            if self.trainer.use_dp or self.trainer.use_ddp2:
                val_acc = torch.mean(val_acc)
            # We weight the batch accuracy by batch size to not give
            # higher weight to the items of a smaller, final bacth.
            batch_size = output['batch_size']
            val_acc_mean += val_acc * batch_size
            total_size += batch_size
        val_loss_mean /= len(outputs)
        val_acc_mean /= total_size
        self.log(test_type+'_loss', val_loss_mean)
        self.log(test_type+'_acc',  val_acc_mean)

    def validation_epoch_end(self, outputs: list) -> None:
        self.test_or_validation_epoch_end('val', outputs)
                                     
    def test_epoch_end(self, outputs: list) -> None:
        self.test_or_validation_epoch_end('test', outputs)
        
    def configure_optimizers(self):
        """ Sets different Learning rates for different parameter groups. """
        parameters = [
            {"params": self.classification_head.parameters()},
            {
                "params": self.bert.parameters(),
                "lr": self.hparams.encoder_learning_rate,
                #"weight_decay": 0.01,  # TODO: try this as it is in the BERT paper
            },
        ]
        optimizer = optim.Adam(parameters, lr=self.hparams.learning_rate)
        return [optimizer], []

    def on_epoch_end(self):
        """ Pytorch lightning hook """
        if self.current_epoch + 1 >= self.nr_frozen_epochs:
            self.unfreeze_encoder()

In [ ]:
xval_run = 0  # 0 to 9


#class DotDict(pl.utilities.AttributeDict):    
#    __getattr__ = dict.get

    # the above misses pickle support; presumable dict defines custom pickle behaviour
    # that causes the unpickled object to be a plain dict object

print('batch_size', batch_size)

classifier = Classifier(
    hparams = {
        "encoder_learning_rate": 1e-05,  # Encoder specific learning rate
        "learning_rate":         3e-05,  # Classification head learning rate
        "nr_frozen_epochs":      3,      # Number of epochs we want to keep the encoder model frozen
        "loader_workers":        4,      # How many subprocesses to use for data loading.
                                         # (0 means that the data will be loaded in the main process)
        "batch_size":            batch_size,
        "gpus":                  1,
    },
    # parameters for SlicedDataModule:
    data_split = splits[xval_run],
    # parameters for SlicedDocument():
    tokeniser                   = tokeniser,
    fraction_for_first_sequence = 0.0,   # set to 0.0001 to duplicate short documents
    max_sequence_length         = max_sequence_length,
    second_part_as_sequence_B   = False,
    preproc_batch_size          = 8
)   
print('Ready.')

batch_size 10
Ready.


In [ ]:
# https://pytorch-lightning.readthedocs.io/en/latest/common/early_stopping.html

from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint

early_stop_callback = EarlyStopping(
    monitor   = 'val_acc',
    min_delta = 0.00,
    patience  = 5,
    verbose   = False,
    mode      = 'max',
)

save_top_model_callback = ModelCheckpoint(
    save_top_k = 3,
    monitor    = 'val_acc',
    mode       = 'max',
    filename   = '{val_acc:.4f}-{epoch:02d}-{val_loss:.4f}'
)

trainer = pl.Trainer(
    callbacks=[early_stop_callback, save_top_model_callback],
    max_epochs = 6,
    min_epochs = classifier.hparams.nr_frozen_epochs + 2,
    gpus = classifier.hparams.gpus,
    accumulate_grad_batches = 4,   # compensate for small batch size
    #limit_train_batches = 10,  # use only a subset of the data during development for higher speed
    check_val_every_n_epoch = 1,
    # https://github.com/PyTorchLightning/pytorch-lightning/issues/6690
    logger = pl.loggers.TensorBoardLogger(os.path.abspath('lightning_logs')),
)
start = time.time()
trainer.fit(classifier, classifier.data)
print('Training time: %.0f minutes' %((time.time()-start)/60.0))

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name                | Type             | Params
---------------------------------------------------------
0 | bert                | BertModel        | 109 M 
1 | classification_head | Sequential       | 1.6 M 
2 | _loss               | CrossEntropyLoss | 0     
---------------------------------------------------------
1.6 M     Trainable params
109 M     Non-trainable params
111 M     Total params
444.230   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Training time: 21 minutes


In [ ]:
# https://www.youtube.com/watch?v=nCq_vy9qE-k at 44:59

%reload_ext tensorboard
%tensorboard --logdir lightning_logs/

In [ ]:
print('The best model is', save_top_model_callback.best_model_path)

print('Best validation set accuracy:', save_top_model_callback.best_model_score)

# The following automatically loads the best weights according to
# https://pytorch-lightning.readthedocs.io/en/latest/common/lightning_module.html

print('Test results via trainer.test():')
results = trainer.test()  # also prints results as a side effect

The best model is /home/jwagner/bert/teaching-CA4023-NLP/CA4023-NLP/notebooks/lightning_logs/default/version_28/checkpoints/val_acc=0.9500-epoch=05-val_loss=0.1351.ckpt
Best validation set accuracy: tensor(0.9500, device='cuda:0')
Test results via trainer.test():


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.8949999213218689, 'test_loss': 0.21454036235809326}
--------------------------------------------------------------------------------


## Save Best Model outside Logs

Rather than manually locating the best model in the lightning logs folder and copying it to another location, use the  library to save a copy. This also gives us the option to save a copy without the training state of the Adam optimiser, reducing model size by about 67%, training parameters and filesystem paths that we may not want to share with users of the model.

In [ ]:
# https://pytorch-lightning.readthedocs.io/en/latest/common/weights_loading.html

# after just having run test(), the best checkpoint is still loaded but that's
# not a documented feature so to be on the safe side for future versions we
# need to explicitly load the best checkpoint:

best_model = Classifier.load_from_checkpoint(
    checkpoint_path = trainer.checkpoint_callback.best_model_path
    # the hparams including hparams.batch_size appear to have been
    # saved in the checkpoint automatically
)
# best_model.save_checkpoint('best.ckpt') does not exist
# --> need to wrap model into trainer to be able to save a checkpoint

new_trainer = pl.Trainer(
    resume_from_checkpoint = trainer.checkpoint_callback.best_model_path,
    gpus = -1,  # avoid warnings (-1 = automatic selection)
    # https://github.com/PyTorchLightning/pytorch-lightning/issues/6690
    logger = pl.loggers.TensorBoardLogger(os.path.abspath('lightning_logs')),
)
new_trainer.model = best_model  # @model.setter in plugins/training_type/training_type_plugin.py

#new_trainer.save_checkpoint("best-model.ckpt")  # contains absoulte paths and training parameters

new_trainer.save_checkpoint(
    "best-model-weights-only.ckpt",
    True,  # save_weights_only
)

# to just save the bert model in pytorch format and without the classification head, we could follow
# https://github.com/PyTorchLightning/pytorch-lightning/issues/3096#issuecomment-686877242
best_model.bert.save_pretrained('best-bert-encoder.pt')

# Since the lightning module inherits from pytorch, we can save the full network in
# pytorch format:
torch.save(best_model.state_dict(), 'best-model.pt')

print('Ready')

GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: None, using: 0 TPU cores
INFO:lightning:TPU available: None, using: 0 TPU cores


Ready


Note: The `.ckpt` files are zip files containing a [pickle](https://docs.python.org/3/library/pickle.html) file, version information and various binary files, presumably numpy arrays.

## Load a Model and Test Again

In [ ]:
best_model = Classifier.load_from_checkpoint(
    checkpoint_path = 'best-model-weights-only.ckpt'
)

best_model.eval()  # enter prediction mode, e.g. turn off dropout

print(best_model.data.data_split)  # confirm the data is not saved

test_dataloader = DataLoader(
    dataset = SlicedDocuments(
        raw_data                    = splits[xval_run][-1],  # test data
        tokeniser                   = tokeniser,
        fraction_for_first_sequence = 0.0,
        max_sequence_length         = max_sequence_length,
        second_part_as_sequence_B   = False,
        preproc_batch_size          = 8
    ),
    batch_size  = best_model.hparams.batch_size,
    collate_fn  = best_model.prepare_sample,
    num_workers = best_model.hparams.loader_workers,
)

print('number of batches:', len(test_dataloader))

new_trainer = pl.Trainer(
    gpus = -1,
    # https://github.com/PyTorchLightning/pytorch-lightning/issues/6690
    logger = pl.loggers.TensorBoardLogger(os.path.abspath('lightning_logs')),
)

if best_model.tokenizer is None:
    print('setting tokeniser')
    best_model.tokenizer = tokeniser

print(new_trainer.test(best_model, test_dataloaders = [test_dataloader]))

(None, None, None)


GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: None, using: 0 TPU cores
INFO:lightning:TPU available: None, using: 0 TPU cores


number of batches: 20
setting tokeniser


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.8949999213218689, 'test_loss': 0.21454036235809326}
--------------------------------------------------------------------------------
[{'test_loss': 0.21454036235809326, 'test_acc': 0.8949999213218689}]


## Make Predictions

Pytorch_lightning does not seem to provide functionality to re-use above code for making predictions. The example code from their website directly calls the `forward()` function of the model, assuming that the inputs of the test items are ready in a suitable batch. For a small test set that does not exceed the batch size, we can manally create such as a batch as follows.

### Small Test Sets

In [ ]:
# reminder: In our dataset, documents are lists of sentences
# and each sentence is a list of words and punctuation

mini_test_set  = [
    # document 1
    ([
        'This movie is great .'.split(),
        'So much fun .'.split(),
    ], 'pos'
    ),
    # document 2
    ([
        'What a waste of time .'.split(),
        'Never seen anything this bad .'.split(),
    ], 'neg'
    ), 
]
dataset = SlicedDocuments(  # subclass of torch.utils.data.Dataset
    mini_test_set,
    preproc_batch_size = 8,
    # the following should match the trained model
    tokeniser = tokeniser,
    fraction_for_first_sequence = 0.0,  
    max_sequence_length = max_sequence_length,
    second_part_as_sequence_B = False,
)

print('model device:', best_model.device)

print('number of documents:', len(dataset))

model device: cuda:0
number of documents: 2


In [ ]:
import numpy

encoded_batch, gold_labels = best_model.prepare_sample(dataset)
print('number of items in batch:', len(encoded_batch))  # TODO: Why is this not len(dataset)?

best_model.eval()  # just in case (already called further above)

best_model.freeze()  # some examples call this before making predictions

# https://github.com/huggingface/transformers/issues/5111
encoded_batch.to(best_model.device)

model_out = best_model(encoded_batch)

# adjsuted copy of code from predict()
logits = model_out["logits"]
logits = torch.Tensor.cpu(logits).numpy()
predicted_labels = [
    best_model.data.label_encoder.index_to_token[prediction]
    for prediction in numpy.argmax(logits, axis=1)
]

print('number of predictions:', len(predicted_labels))  # matches len(dataset)

for index, item in enumerate(dataset):
    print('[%d]' %index, item)
    print('prediction:', predicted_labels[index])
    
# the 'parts' list has two parts when second_part_as_sequence_B = True and fraction_for_first_sequence > 0.0

number of items in batch: 4
number of predictions: 2
[0] {'parts': [['This', 'movie', 'is', 'great', '.', 'So', 'much', 'fun', '.']], 'label': 'pos'}
prediction: pos
[1] {'parts': [['What', 'a', 'waste', 'of', 'time', '.', 'Never', 'seen', 'anything', 'this', 'bad', '.']], 'label': 'neg'}
prediction: neg


### Large Test Sets
For test sets that do not fit into a single batch, we extend the model's evaluation function to also record predictions in the metrics dictionary. We keep a record of the inputs as well as the test items may be distributed over multiple GPUs and the order of items may therefore change. We then only need to tokenise the test items again and fetch the predictions from the metrics dictionary.

In [ ]:
# uses best_model, dataset and new_trainer from above

best_model.start_recording_predictions()

new_trainer.test(best_model, test_dataloaders = [DataLoader(
    dataset     = dataset,   # let's first test the functionality with a small test set
    batch_size  = best_model.hparams.batch_size,
    collate_fn  = best_model.prepare_sample,
    num_workers = best_model.hparams.loader_workers,
)])

best_model.stop_recording_predictions()

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 1.0, 'test_loss': 0.01612304151058197}
--------------------------------------------------------------------------------


In [ ]:
print(best_model.seq2label)

{(101, 2023, 3185, 2003, 2307, 1012, 2061, 2172, 4569, 1012, 102): 'pos', (101, 2054, 1037, 5949, 1997, 2051, 1012, 2196, 2464, 2505, 2023, 2919, 1012, 102): 'neg'}


In [ ]:
for index, item in enumerate(dataset):
    print('[%d]' %index, item)
    input_token_ids = best_model.prepare_sample([item])[0]['input_ids']
    key = input_token_ids.tolist()[0]
    # truncate zeros
    while key and key[-1] == 0:
        del key[-1]
    key = tuple(key)
    try:
        print('prediction:', best_model.seq2label[key])
    except KeyError:
        print('prediction not found')

[0] {'parts': [['This', 'movie', 'is', 'great', '.', 'So', 'much', 'fun', '.']], 'label': 'pos'}
prediction: pos
[1] {'parts': [['What', 'a', 'waste', 'of', 'time', '.', 'Never', 'seen', 'anything', 'this', 'bad', '.']], 'label': 'neg'}
prediction: neg


In [ ]:
# now with a bigger dataset

best_model.start_recording_predictions()

xval_test_dataset = SlicedDocuments(
    raw_data                    = splits[xval_run][-1],  # test data
    tokeniser                   = tokeniser,
    fraction_for_first_sequence = 0.0,
    max_sequence_length         = max_sequence_length,
    second_part_as_sequence_B   = False,
    preproc_batch_size          = 8
)

new_trainer.test(best_model, test_dataloaders = [DataLoader(
    dataset     = xval_test_dataset,    
    batch_size  = best_model.hparams.batch_size,
    collate_fn  = best_model.prepare_sample,
    num_workers = best_model.hparams.loader_workers,
)])

best_model.stop_recording_predictions()
print('Ready')

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.8949999213218689, 'test_loss': 0.21454036235809326}
--------------------------------------------------------------------------------
Ready


In [ ]:
print('\t'.join(['index', 'gold', 'prediction', 'correct', 'text']))
for index, item in enumerate(xval_test_dataset):
    row = []
    row.append('[%d]' %index)
    row.append(item['label'])
    input_token_ids = best_model.prepare_sample([item])[0]['input_ids']
    key = input_token_ids.tolist()[0]
    # truncate zeros
    while key and key[-1] == 0:
        del key[-1]
    key = tuple(key)
    try:
        prediction = best_model.seq2label[key]
    except KeyError:
        prediction = 'unknown'
    row.append(prediction)
    if prediction == item['label']:
        row.append('yes')
    else:
        row.append('no')
    row.append(' '.join(item['parts'][0]))
    print('\t'.join(row))

index	gold	prediction	correct
[0]	pos	pos	✓
[1]	pos	neg	x
[2]	pos	pos	✓
[3]	pos	pos	✓
[4]	pos	pos	✓
[5]	pos	pos	✓
[6]	pos	pos	✓
[7]	pos	pos	✓
[8]	pos	pos	✓
[9]	pos	pos	✓
